In [1]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh

from tabulate import tabulate
import json, time, datetime
import telegram

pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# !pip install pandas requests lxml beautifulsoup4 python-telegram-bot tabulate --upgrade

# Telegram API

In [3]:
class TelegramMessenger:
    """
    https://forums.fast.ai/t/training-metrics-as-notifications-on-mobile-using-callbacks/17330/4

    Utilizes this API Library:
       https://github.com/python-telegram-bot/python-telegram-bot
    To install:
       pip install python-telegram-bot --upgrade

    {"api_key": "462203107:<your API key>",
     "chat_id": "<your chat ID>"}

    Here's how you get an API key:
       https://core.telegram.org/api/obtaining_api_id
    Here's how you get your chat ID:
       https://stackoverflow.com/questions/32423837/telegram-bot-how-to-get-a-group-chat-id

    """

    def __init__(self, cred_file_path):
        self.__credentials = json.loads(open(cred_file_path).read())
        # Initialize bot
        self.bot = telegram.Bot(token=self.__credentials['api_key'])

    def send_message(self, message='Done'):
        self.bot.send_message(parse_mode='HTML', chat_id=self.__credentials['chat_id'], text=message)

# Global data

In [24]:
def convert1(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

def get_data():
    url = 'https://www.worldometers.info/coronavirus'
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//*[@id="main_table_countries_today"]/tbody[1]')
    rows = []
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        row = [x.text_content() for x in t.findall('td')[:-1]]
        rows.append(row)

    columns = ['Country', 'Cases', 'New', 'Deaths',
               'New Deaths', 'Recovered', 'Active', 'x', 'x', 'x', 'x', 'x']
    df = pd.DataFrame(rows, columns=columns).iloc[:, :5].fillna(0)
    df = df.replace(' ', '').replace(',', '', regex=True).replace('', 0)
    df.dropna(inplace=True)

    for col in df.columns[1:]:
        df[col] = df[col].replace('+', '').apply(pd.to_numeric).astype(int)

    df.Country = df.Country.apply(lambda x: x.replace('\n', ''))

    return df.sort_values(['New', 'Cases'], ascending=False)

get_data()

,Country,Cases,New,Deaths,New Deaths
7,World,1910283,58026,118546,4352
1,Europe,904862,23752,79570,2802
0,North America,617639,19344,24566,1090
8,USA,577729,17429,23115,1010
2,Asia,309581,10845,11331,329
13,UK,88621,4342,11329,717
11,France,136779,4188,14967,574
15,Turkey,61049,4093,1296,98
3,South America,53826,3245,2163,82
10,Italy,159516,3153,20465,566


# India data

In [11]:
def get_newcases_time_series():
    url = 'https://api.covid19india.org/data.json'
    page = requests.get(url)
    df = pd.DataFrame(json.loads(page.content)[
                      'cases_time_series']).dropna().set_index('date')
    df = df.iloc[-10:, :1]
    # df.loc["Total"] = df.sum()
    df.columns = ['NewCases']
    df = df.astype(int)
    df['%Change'] = df.NewCases.pct_change()*100
    df = df[-7:]
    df['%Change'] = df['%Change'].astype(int).round(2).apply(lambda x: f'{x}%')
    return df

get_newcases_time_series()

,NewCases,%Change
date,,
22 March,69,-9%
23 March,102,47%
24 March,66,-35%
25 March,86,30%
26 March,78,-9%
27 March,151,93%
28 March,143,-5%


In [12]:
def get_total_time_series():
    url = 'https://api.covid19india.org/data.json'
    page = requests.get(url)
    df = pd.DataFrame(json.loads(page.content)['cases_time_series']).dropna().set_index('date')
    df = df[['totalconfirmed']].rename({'totalconfirmed': 'TotalCases'}, axis=1)
    df = df.iloc[-10:,:1]
    df = df.astype(int)
    df['%Increase'] = df.TotalCases.pct_change()*100
    df = df[-7:]
    df['%Increase'] = df['%Increase'].astype(int).round(2).apply(lambda x: f'{x}%')
    return df
    
get_total_time_series()

,TotalCases,%Increase
date,,
22 March,403,20%
23 March,505,25%
24 March,571,13%
25 March,657,15%
26 March,735,11%
27 March,886,20%
28 March,1029,16%


In [3]:
def get_data():
    url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSz8Qs1gE_IYpzlkFkCXGcL_BqR8hZieWVi-rphN1gfrO3H4lDtVZs4kd0C3P8Y9lhsT1rhoB-Q_cP4/pubhtml#'
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//*[@id="1896310216"]/div/table/tbody')
    rows=[]
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        row = [x.text_content() for x in t.findall('td')[:-1]]
        rows.append(row)

    df = pd.DataFrame(rows[1:], columns=rows[0]).replace('', None).dropna().drop_duplicates()
    df[['Confirmed', 'Recovered', 'Deaths', 'Active']] = df[['Confirmed', 'Recovered', 'Deaths', 'Active']].apply(pd.to_numeric)
    return df

df = get_data()
print(len(df))
df.head()

38


,State,,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code,Delta_Confirmed,Delta_Recovered,Delta_Deaths
0,Total,,13430,1768,448,11214,16/04/2020 23:43:05,TT,1059,259,26
2,Maharashtra,,3202,300,194,2708,16/04/2020 21:33:06,MH,286,5,7
3,Delhi,,1640,52,38,1550,16/04/2020 23:23:06,DL,62,11,6
4,Tamil Nadu,,1267,180,15,1072,15/04/2020 18:40:25,TN,25,62,1
5,Madhya Pradesh,,1164,70,55,1039,16/04/2020 21:33:07,MP,226,6,2


In [92]:
# df_new = df.copy()
# df_new.Confirmed = list(np.random.randint(50, size=10)) + df.Confirmed.to_list()[10:38]
# df_new = df_new.sample(frac=1)
# df_new.head()

In [9]:
def get_clean_table(df):
    message = tabulate(df, headers='keys',
                       tablefmt='simple', numalign="center")
    return '<pre>' + message + '</pre>'

try:
    bot = TelegramMessenger('../india-config.json')
except:
    bot = TelegramMessenger('india-config.json')

In [95]:
curr_date = datetime.datetime.now().date()

while True:
    df_new = get_data()
    df_update = df_new
#     df_update['old_confirmed'] = df.Confirmed
    df_update = df_update.merge(df.rename({'Confirmed': 'old_confirmed'}, axis=1))
    df_update['New'] = df_update['Confirmed'] - df_update['old_confirmed']
    df_update = df_update[df_update['New']>0]
    print(df_update)
    df_update = df_update[['State', 'New', 'Confirmed', 'Deaths']].set_index('State')
    curr_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    curr_time = f'Case update at: {curr_time}'
    print(curr_time)
    if len(df_update)>0:
        message = get_clean_table(df_update)
        print(message)
        bot.send_message(curr_time)
        bot.send_message(message)
    else:
        print('No new cases')
    date = datetime.datetime.now().date()
    if date != curr_date:
        df = df_new
#     break
    time.sleep(600)

Empty DataFrame
Columns: [State, , Confirmed, Recovered, Deaths, Active, old_confirmed, New]
Index: []
Case update at: 2020-03-23 23:31
No new cases


KeyboardInterrupt: 

In [393]:
# !jupyter nbconvert --to script telegram.ipynb

[NbConvertApp] Converting notebook telegram.ipynb to script
[NbConvertApp] Writing 3206 bytes to telegram.py


In [23]:
bot = TelegramMessenger('../test-config.json')

In [24]:
df = get_data()

In [25]:
df = df[['Country', 'New', 'Total', 'Deaths']]
df.Country = df.Country.apply(lambda x: x[:8])
df = df.rename({'Country': 'Count','Total':'Conf'}, axis=1).set_index('Count')

In [26]:
# .replace(' | ', '|').replace('------|', '|')
message = '<pre>' + tabulate(df[:60], headers='keys', tablefmt='simple', numalign="center") + '</pre>'
bot.send_message(message)

In [76]:
# url = f'https://api.telegram.org/bot11352/getUpdates'
# requests.get(url).text

In [33]:
# for g, sub_df in df.groupby(np.arange(len(df)) // 40):
#     print(sub_df.shape)

In [10]:
bot.send_message(get_clean_table(get_time_series()))